In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# %%
import torch
import string

In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.2MB 7.1MB/s 


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.5MB 8.9MB/s 
     |████████████████████████████████| 3.3MB 67.4MB/s 
     |████████████████████████████████| 901kB 68.3MB/s 


In [ ]:
cd drive/My Drive/Colab Notebooks/experiments

/content/drive/My Drive/Colab Notebooks/experiments


In [ ]:
#import pandas as pd
#text = pd.read_csv("stockholm/bert_code/trofi_bert_subs/trofi_tp.csv")

In [ ]:
#text.shape

(144, 5)

In [ ]:
#text.to_csv(r'trofi_bert_tp.txt', header=None, index=None, sep=' ', mode='a')

In [ ]:
with open('stockholm/mlm/preds/trofi_bert_tp.txt', 'r') as fp:
    text = fp.read().split('\n')

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertForMaskedLM.from_pretrained('bert-base-uncased').eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from transformers import XLNetTokenizer, XLNetLMHeadModel
xlnet_tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
xlnet_model = XLNetLMHeadModel.from_pretrained('xlnet-base-cased').eval()

In [ ]:
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM
xlmroberta_tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
xlmroberta_model = XLMRobertaForMaskedLM.from_pretrained('xlm-roberta-base').eval()

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large').eval()

In [ ]:
from transformers import ElectraTokenizer, ElectraForMaskedLM
electra_tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-generator')
electra_model = ElectraForMaskedLM.from_pretrained('google/electra-small-generator').eval()

In [ ]:
from transformers import RobertaTokenizer, RobertaForMaskedLM
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = RobertaForMaskedLM.from_pretrained('roberta-base').eval()

In [ ]:
top_k = 10

In [ ]:
def decode(tokenizer, pred_idx, top_clean):
    ignore_tokens = string.punctuation + '[PAD]'
    tokens = []
    for w in pred_idx:
        token = ''.join(tokenizer.decode(w).split())
        if token not in ignore_tokens:
            tokens.append(token.replace('##', ''))
    return '\n'.join(tokens[:top_clean])

In [ ]:
def encode(tokenizer, text_sentence, add_special_tokens=True):
    text_sentence = text_sentence.replace('<mask>', tokenizer.mask_token)
    # if <mask> is the last token, append a "." so that models dont predict punctuation.
    if tokenizer.mask_token == text_sentence.split()[-1]:
        text_sentence += ' .'

    input_ids = torch.tensor([tokenizer.encode(text_sentence, add_special_tokens=add_special_tokens)])
    mask_idx = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]
    return input_ids, mask_idx

In [ ]:
def get_all_predictions(text_sentence, top_clean=5):
    # ========================= BERT =================================
    print(text_sentence)
    input_ids, mask_idx = encode(bert_tokenizer, text_sentence)
    with torch.no_grad():
        predict = bert_model(input_ids)[0]
    bert = decode(bert_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= XLNET LARGE =================================
    input_ids, mask_idx = encode(xlnet_tokenizer, text_sentence, False)
    perm_mask = torch.zeros((1, input_ids.shape[1], input_ids.shape[1]), dtype=torch.float)
    perm_mask[:, :, mask_idx] = 1.0  # Previous tokens don't see last token
    target_mapping = torch.zeros((1, 1, input_ids.shape[1]), dtype=torch.float)  # Shape [1, 1, seq_length] => let's predict one token
    target_mapping[0, 0, mask_idx] = 1.0  # Our first (and only) prediction will be the last token of the sequence (the masked token)

    with torch.no_grad():
        predict = xlnet_model(input_ids, perm_mask=perm_mask, target_mapping=target_mapping)[0]
    xlnet = decode(xlnet_tokenizer, predict[0, 0, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= XLM ROBERTA BASE =================================
    input_ids, mask_idx = encode(xlmroberta_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = xlmroberta_model(input_ids)[0]
    xlm = decode(xlmroberta_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= BART =================================
    input_ids, mask_idx = encode(bart_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = bart_model(input_ids)[0]
    bart = decode(bart_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= ELECTRA =================================
    input_ids, mask_idx = encode(electra_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = electra_model(input_ids)[0]
    electra = decode(electra_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= ROBERTA =================================
    input_ids, mask_idx = encode(roberta_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = roberta_model(input_ids)[0]
    roberta = decode(roberta_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    return {'bert': bert,
            'xlnet': xlnet,
            'xlm': xlm,
            'bart': bart,
            'electra': electra,
            'roberta': roberta}

In [ ]:
text[:35]

['"Unfortunately , they wo n\'t work in the harbor tunnel where business travelers sometimes get stuck on the way in from the airport . . . " 1 stick 15 1',
 '"MISSED CONNECTIONS : If you \'re bound for Denver but get stuck in Detroit for the night because of a missed connection , do n\'t expect the airline to help " 1 stick 11 1',
 '"It also vindicates Pratt \'s efforts to fix problems in customer service that contributed to its loss of dominance in the commercial jet engine field to General Electric Co " 1 fix 7 1',
 '"A surprisingly strong October employment report , implying that the economy is n\'t cooling off as fast as many investors had hoped , could force a sharp retreat in stock prices today , he said " 1 cool 13 1',
 '"One look at her pudgy face , shot to make her seem like a powder - brained Kewpie doll , vaporizes whatever credibility Jamie \'s anguish might have had " 1 vaporize 20 1',
 '"Weeks later , \'\' Penthouse finally realized that Dominion was intentionally destro

In [ ]:
text_sentence = 'Unfortunately , they wo n\'t work in the harbor tunnel where business travelers sometimes get <mask> on the way in from the airport . . . '
get_all_predictions(text_sentence)

Unfortunately , they wo n't work in the harbor tunnel where business travelers sometimes get <mask> on the way in from the airport . . . 


{'bart': 'stuck\nlost\na\ninto\nhit',
 'bert': 'stuck\nlost\nstranded\noff\ncaught',
 'electra': 'off\nout\nlost\naway\nup',
 'roberta': 'stuck\nlost\nstranded\nhit\ncaught',
 'xlm': 'stuck\nlost\ncaught\nhit\nseen',
 'xlnet': 'lost\nget\nstuck\ndrowned\nout'}

In [ ]:
text_sentence = "MISSED CONNECTIONS : If you \'re bound for Denver but get <mask> in Detroit for the night because of a missed connection , do n\'t expect the airline to help "
get_all_predictions(text_sentence)

MISSED CONNECTIONS : If you 're bound for Denver but get <mask> in Detroit for the night because of a missed connection , do n't expect the airline to help 


{'bart': 'stuck\nstranded\ndelayed\nput\ndiverted',
 'bert': 'stuck\nback\noff\nstranded\ndown',
 'electra': 'up\nback\nout\nstuck\nlost',
 'roberta': 'stuck\nstranded\ntrapped\ncaught\ndelayed',
 'xlm': 'stuck\nlost\nback\nout\nup',
 'xlnet': 'stuck\nstranded\nback\nlost\nmissed'}

In [ ]:
text_sentence = "It also vindicates Pratt \'s efforts to <mask> problems in customer service that contributed to its loss of dominance in the commercial jet engine field to General Electric Co "
get_all_predictions(text_sentence)

It also vindicates Pratt 's efforts to <mask> problems in customer service that contributed to its loss of dominance in the commercial jet engine field to General Electric Co 


{'bart': 'repair\nfix\novercome\naddress\ncorrect',
 'bert': 'address\nresolve\nsolve\nfix\ncorrect',
 'electra': 'solve\naddress\novercome\nfix\nresolve',
 'roberta': 'fix\naddress\ncorrect\nresolve\nremedy',
 'xlm': 'address\nsolve\nresolve\nfix\ncorrect',
 'xlnet': 'solve\nto\naddress\nfix\nthe'}

In [ ]:
text_sentence = "A surprisingly strong October employment report , implying that the economy is n\'t <mask> off as fast as many investors had hoped , could force a sharp retreat in stock prices today , he said "
get_all_predictions(text_sentence)

A surprisingly strong October employment report , implying that the economy is n't <mask> off as fast as many investors had hoped , could force a sharp retreat in stock prices today , he said 


{'bart': 'cooling\nslowing\nreally\ntaking\nbeing',
 'bert': 'taking\ncooling\ngoing\ngetting\ntake',
 'electra': 'paying\ncoming\ngoing\nbuying\nslowing',
 'roberta': 'cooling\ntaking\neasing\ncoming\nfalling',
 'xlm': 'going\npaying\ntaking\ncoming\ngetting',
 'xlnet': 'quite\nreally\ngetting\ncoming\npulling'}

In [ ]:
text_sentence = "One look at her pudgy face , shot to make her seem like a powder - brained Kewpie doll , <mask> whatever credibility Jamie \'s anguish might have had "
get_all_predictions(text_sentence)

One look at her pudgy face , shot to make her seem like a powder - brained Kewpie doll , <mask> whatever credibility Jamie 's anguish might have had 


{'bart': 'and\nwas\nmade\nwould\nthe',
 'bert': 'destroyed\nruined\ntook\nshattered\ngave',
 'electra': 'showed\nand\nshows\nrevealed\nreveals',
 'roberta': 'erased\ndestroys\ndestroyed\nshattered\nundermined',
 'xlm': 'with\nand\nfor\nor\ngiven',
 'xlnet': 'and\nany\nin\nwithout\nto'}

In [ ]:
text_sentence = "Weeks later , \'\' Penthouse finally realized that Dominion was intentionally <mask> the deal , \'\' sought other financing , failed , and had to abandon the project , the judge said "
get_all_predictions(text_sentence)

Weeks later , '' Penthouse finally realized that Dominion was intentionally <mask> the deal , '' sought other financing , failed , and had to abandon the project , the judge said 


{'bart': 'trying\nmisleading\nlying\nwithholding\nhiding',
 'bert': 'violating\ndelaying\nbreaking\nblowing\nblocking',
 'electra': 'blocking\nbreaking\nabandoning\nruining\nviolating',
 'roberta': 'rigging\nbreaking\nkilling\nmanipulating\ncutting',
 'xlm': 'breaking\nmaking\nin\nselling\ntaking',
 'xlnet': 'in\nbehind\ninto\non\nfinancing'}

In [ ]:
text_sentence = "It recently sued a small local factory that was knocking off its <mask> - off with a game called Entrepreneurs"
get_all_predictions(text_sentence)

It recently sued a small local factory that was knocking off its <mask> - off with a game called Entrepreneurs


{'bart': 'logo\nproducts\nname\nproduct\nproduction',
 'bert': 'pay\nsell\ntrade\nspin\nrun',
 'electra': 'spin\nrip\nknock\ntake\nsell',
 'roberta': 'knock\nhands\nbake\nrip\npay',
 'xlm': 'business\nworkers\ncustomers\nemployees\ncompetition',
 'xlnet': 'own\nand\nits\n‘\nagain'}

In [ ]:
text_sentence = "It <mask> into the vacuum created by the collapse of a Keynesian mainstream that had no explanation for simultaneous stagnation and inflation "
get_all_predictions(text_sentence)

It <mask> into the vacuum created by the collapse of a Keynesian mainstream that had no explanation for simultaneous stagnation and inflation 


{'bart': 'was\nis\nalso\nwould\nhad',
 'bert': 'fell\nlooked\nemerged\nsank\nplunged',
 'electra': 'fell\nplunged\nwent\ncame\nbroke',
 'roberta': 'fell\nstepped\ntapped\nfits\ndisappeared',
 'xlm': 'fell\nwent\ncame\nsank\nran',
 'xlnet': 'In\nin\nIt\nwas\ninto'}

In [ ]:
text_sentence = "The report <mask> bullish sentiment brought about by a smaller Brazilian crop this year , analysts said "
get_all_predictions(text_sentence)

The report <mask> bullish sentiment brought about by a smaller Brazilian crop this year , analysts said 


{'bart': 'also\ncould\nwas\nis\nadded',
 'bert': 'cited\nnoted\nblamed\nhighlighted\nreported',
 'electra': 'reflects\nreflected\nrevealed\nshowed\nhighlighted',
 'roberta': 'reflects\nreflected\nunderscores\nhighlights\nsupports',
 'xlm': 'showed\nshows\nfound\ndescribed\nreported',
 'xlnet': 'report\nof\nreported\nwas\nreports'}

In [ ]:
text_sentence = "The stock has risen more than 17 % in less than two weeks on <mask> up volume "
get_all_predictions(text_sentence)

The stock has risen more than 17 % in less than two weeks on <mask> up volume 


{'bart': 'the\nstrong\na\nhopes\nheavy',
 'bert': 'start\nthe\nline\nroll\ncall',
 'electra': 'a\nan\npicking\nthe\ngrowing',
 'roberta': 'pent\nbid\nstepped\nheavy\npick',
 'xlm': 'the\nan\nits\na\nstraight',
 'xlnet': 'on\nan\nthe\nincreased\nboth'}

In [ ]:
text_sentence = "It is to correct for such gaps in market operations -LRB- or \'\' externalities , \'\' as economists call them -RRB- that governments have a responsibility to step in and <mask> a helping hand "
get_all_predictions(text_sentence)

It is to correct for such gaps in market operations -LRB- or '' externalities , '' as economists call them -RRB- that governments have a responsibility to step in and <mask> a helping hand 


{'bart': 'lend\nprovide\ngive\noffer\nhelp',
 'bert': 'provide\nlend\ngive\nbe\noffer',
 'electra': 'have\ntake\nget\ngive\nput',
 'roberta': 'lend\noffer\nextend\nprovide\ngive',
 'xlm': 'provide\noffer\ngive\nbe\nget',
 'xlnet': 'lend\noffer\nextend\ngive\na'}

In [ ]:
text_sentence = "But Madison Avenue \'s interest in Mr. Tyson is <mask> because of the barrage of negative publicity about an alleged suicide attempt and his stormy marriage to actress Robin Givens "
get_all_predictions(text_sentence)

But Madison Avenue 's interest in Mr. Tyson is <mask> because of the barrage of negative publicity about an alleged suicide attempt and his stormy marriage to actress Robin Givens 


{'bart': 'likely\nwaning\nin\nnot\nunlikely',
 'bert': 'strained\nlimited\ndiminished\nrevived\nrestricted',
 'electra': 'growing\nlimited\nprimarily\nstrong\nlargely',
 'roberta': 'waning\nlimited\nfading\nunderstandable\nunusual',
 'xlm': 'growing\nincreasing\nlost\nlimited\nnot',
 'xlnet': 'not\nonly\njust\nbecause\nprimarily'}

In [ ]:
text_sentence = "After years of being out of work and on welfare , many of the unemployed now lack the ability or initiative to <mask> openings "
get_all_predictions(text_sentence)

After years of being out of work and on welfare , many of the unemployed now lack the ability or initiative to <mask> openings 


{'bart': 'find\nlook\ntake\nkeep\nget',
 'bert': 'seek\nmake\naccept\nattend\npursue',
 'electra': 'find\nmake\nget\ncreate\nreach',
 'roberta': 'fill\nfind\nseek\ncreate\nmake',
 'xlm': 'fill\nopen\nfind\npursue\nsecure',
 'xlnet': 'to\njob\nmake\nopen\naccess'}

In [ ]:
text_sentence = "She \'s been able to <mask> to this policy , largely because of the nature of her music "
get_all_predictions(text_sentence)

She 's been able to <mask> to this policy , largely because of the nature of her music 


{'bart': 'maintain\nremain\navoid\nhold\ncontinue',
 'bert': 'adhere\nstick\nagree\nadapt\nconform',
 'electra': 'adhere\nstick\nadapt\nconform\nagree',
 'roberta': 'adhere\nstick\nadapt\nconform\nappeal',
 'xlm': 'stick\ncontribute\nagree\nconform\nadapt',
 'xlnet': 'come\nget\nto\ngo\naccess'}

In [ ]:
text_sentence = "They also became a major element in the more - than - doubling of the Nikkei average in the two years to last October because of the huge amount of money they <mask> into the market "
get_all_predictions(text_sentence)

They also became a major element in the more - than - doubling of the Nikkei average in the two years to last October because of the huge amount of money they <mask> into the market 


{'bart': 'pumped\nwere\nput\npoured\nmanaged',
 'bert': 'put\nbrought\npoured\ninjected\ninvested',
 'electra': 'put\npoured\nbrought\nsent\npumped',
 'roberta': 'pumped\npoured\ninjected\nput\nbrought',
 'xlm': 'put\nbrought\ntook\ninject\npump',
 'xlnet': 'put\npumped\npoured\ninjected\nbrought'}

In [ ]:
text_sentence = "Some say interest rates may have to rise further to <mask> domestic demand , while others say the problem is more deep - rooted and requires greater investment by United Kingdom industry "
get_all_predictions(text_sentence)

Some say interest rates may have to rise further to <mask> domestic demand , while others say the problem is more deep - rooted and requires greater investment by United Kingdom industry 


{'bart': 'stimulate\nsupport\nboost\nhelp\nspur',
 'bert': 'meet\nsatisfy\nmatch\naccommodate\nreduce',
 'electra': 'meet\nboost\nsupport\nfeed\nsatisfy',
 'roberta': 'stimulate\nboost\nsupport\nmeet\nspur',
 'xlm': 'meet\nreflect\nsupport\naddress\nreduce',
 'xlnet': 'meet\nsatisfy\nto\naccommodate\nrespond'}

In [ ]:
text_sentence = "By 9 a.m . , the cadets are off to class , batlike figures in dark , <mask> rain capes , black briefcases clutched with precision in their left hands "
get_all_predictions(text_sentence)

By 9 a.m . , the cadets are off to class , batlike figures in dark , <mask> rain capes , black briefcases clutched with precision in their left hands 


{'bart': 'black\ndark\nwhite\nhood\nlong',
 'bert': 'black\ngray\nflowing\nlong\nheavy',
 'electra': 'black\nblue\nwhite\ndark\nwet',
 'roberta': 'black\nblue\nshiny\nheavy\nred',
 'xlm': 'black\nwhite\nblue\ndark\nred',
 'xlnet': 'black\nwhite\nred\ngrey\nin'}

In [ ]:
text_sentence = "The Soviet Union is unwilling and unlikely to <mask> the deadline for withdrawing all its troops from Afghanistan , despite its latest blustering and military maneuvering there , Western and Soviet officials indicated "
get_all_predictions(text_sentence)

The Soviet Union is unwilling and unlikely to <mask> the deadline for withdrawing all its troops from Afghanistan , despite its latest blustering and military maneuvering there , Western and Soviet officials indicated 


{'bart': 'extend\nmeet\nchange\npostpone\ndelay',
 'bert': 'meet\naccept\nextend\nset\nreach',
 'electra': 'meet\nreach\nset\nmiss\npass',
 'roberta': 'meet\nmiss\nextend\ndelay\naccept',
 'xlm': 'meet\nreach\nmiss\nhit\nachieve',
 'xlnet': 'meet\nignore\naccept\nto\navoid'}

In [ ]:
text_sentence = "\'\' I found myself <mask> out on the steps in boot camp , \'\' he recalls "
get_all_predictions(text_sentence)

'' I found myself <mask> out on the steps in boot camp , '' he recalls 


{'bart': 'in\ncrying\ngetting\nwith\non',
 'bert': 'passed\nhiding\nhanging\nbleeding\nsprawled',
 'electra': 'hanging\ngoing\nwalking\ncamping\nlaying',
 'roberta': 'passed\nhanging\ncrying\nsitting\ncamping',
 'xlm': 'walking\nrunning\nstanding\ngoing\nmissing',
 'xlnet': 'yourself\nway\nout\nback\nmyself'}

In [ ]:
text_sentence = "Last month , Thiokol said it settled the last suit with the estates of the seven crew members <mask> in the Challenger "
get_all_predictions(text_sentence)

Last month , Thiokol said it settled the last suit with the estates of the seven crew members <mask> in the Challenger 


{'bart': 'who\nkilled\nof\nand\ninvolved',
 'bert': 'involved\naboard\nparticipating\nkilled\nlost',
 'electra': 'involved\nworking\nliving\nremaining\noperating',
 'roberta': 'killed\ninvolved\ninjured\nlost\nmissing',
 'xlm': 'involved\nkilled\noperating\nworking\npresent',
 'xlnet': 'involved\nin\nkilled\nwho\ninjured'}

In [ ]:
text_sentence = "Incumbent Spyros Kyprianou was <mask> out in voting last week "
get_all_predictions(text_sentence)

Incumbent Spyros Kyprianou was <mask> out in voting last week 


{'bart': 'narrowly\nthe\nknocked\nforced\nvoted',
 'bert': 'knocked\nvoted\nleft\nedged\nstill',
 'electra': 'knocked\nruled\nwiped\nshut\nnot',
 'roberta': 'shut\nwiped\nknocked\nedged\nvoted',
 'xlm': 'missing\nnot\nall\nmissed\nsold',
 'xlnet': 'was\nout\nalso\nnot\nhimself'}

In [ ]:
text_sentence = "Those progressive cities now building or planning transit systems for the next century are not viewing the scene from an ivory tower but realize , in a practical sense , that better transit is vital not only for the millions of Americans who currently <mask> onto a transit bus or train over 8.3 billion times each year , but also to their future citizens as well "
get_all_predictions(text_sentence)

Those progressive cities now building or planning transit systems for the next century are not viewing the scene from an ivory tower but realize , in a practical sense , that better transit is vital not only for the millions of Americans who currently <mask> onto a transit bus or train over 8.3 billion times each year , but also to their future citizens as well 


{'bart': 'use\nrely\nlive\ncommute\nride',
 'bert': 'get\nstep\nhop\nmove\nload',
 'electra': 'get\nride\nlog\ngo\njump',
 'roberta': 'step\nhop\nget\njump\npile',
 'xlm': 'hop\ngo\ncycle\nturn\nmove',
 'xlnet': 'hop\nonto\nhold\nride\ngrab'}

In [ ]:
text_sentence = "Seagram must decide by Feb. 9 whether to come up with a higher offer that would <mask> Grand Met out of the bidding war and frustrate its expansion strategy "
get_all_predictions(text_sentence)

Seagram must decide by Feb. 9 whether to come up with a higher offer that would <mask> Grand Met out of the bidding war and frustrate its expansion strategy 


{'bart': 'knock\nforce\nkeep\ntake\nleave',
 'bert': 'keep\nhelp\ntake\nget\nbring',
 'electra': 'be\nget\nmake\nkeep\nhave',
 'roberta': 'lock\nknock\ntake\nkeep\nshut',
 'xlm': 'get\ntake\nkeep\nkick\nbring',
 'xlnet': 'pull\nkeep\ndraw\nwithdraw\nthrow'}

In [ ]:
text_sentence = "Charity officials say they need trustees who are willing to <mask> up their sleeves and work for the organization and who will back up their commitment with donations "
get_all_predictions(text_sentence)

Charity officials say they need trustees who are willing to <mask> up their sleeves and work for the organization and who will back up their commitment with donations 


{'bart': 'roll\nwork\nput\nboth\nsit',
 'bert': 'roll\nlift\nclimb\nthrow\nwear',
 'electra': 'roll\ngive\ntie\ncut\nback',
 'roberta': 'roll\nopen\nrolled\nzip\npush',
 'xlm': 'take\nroll\npull\nlift\ngive',
 'xlnet': 'go\nput\ntake\ncome\nup'}

In [ ]:
text_sentence = "This provides greatly enhanced trouble - shooting capabilities when something does go wrong , and can even be used to automatically switch in a good spare card if something does <mask> "
get_all_predictions(text_sentence)

This provides greatly enhanced trouble - shooting capabilities when something does go wrong , and can even be used to automatically switch in a good spare card if something does <mask> 


{'bart': 'go\nhappen\nnot\nbreak\nindeed',
 'bert': 'happen\nfail\noccur\nnot\nwrong',
 'electra': 'happen\ngo\nwork\nwrong\nnot',
 'roberta': 'go\nhappen\nget\noccur\nbreak',
 'xlm': 'fail\nhappen\nnot\ncrash\nwork',
 'xlnet': 'go\nhappen\nnot\ndo\nsomething'}

In [ ]:
text_sentence = "The Japanese carrier was shaken in 1985 by a crash of a JAL Boeing 747 that <mask> 520 people , the worst single - aircraft accident in aviation history , and safety is a central issue in JAL \'s current negotiations with several unions"
get_all_predictions(text_sentence)

The Japanese carrier was shaken in 1985 by a crash of a JAL Boeing 747 that <mask> 520 people , the worst single - aircraft accident in aviation history , and safety is a central issue in JAL 's current negotiations with several unions


{'bart': 'killed\nclaimed\ndestroyed\ntook\nleft',
 'bert': 'killed\ninjured\nkilling\nlost\nwounded',
 'electra': 'killed\ninjured\nwounded\nleft\naffected',
 'roberta': 'killed\ninjured\nclaimed\nhurt\nkilling',
 'xlm': 'killed\nlost\ninvolved\nhurt\ntook',
 'xlnet': 'killed\ninjured\nclaimed\nkilling\ndied'}

In [ ]:
text_sentence = "It is \'\' Hotlanta \'\' : capital city of the South at a time when the South has risen ; city of myriad services at a time when the service economy rules ; city of the 1980s , more suburban than downtown , more footloose than <mask>"
get_all_predictions(text_sentence)

It is '' Hotlanta '' : capital city of the South at a time when the South has risen ; city of myriad services at a time when the service economy rules ; city of the 1980s , more suburban than downtown , more footloose than <mask>


{'bart': 'the\ndowntown\nupt\ncity\nhip',
 'bert': 'downtown\ncity\nurban\nuptown\nmidtown',
 'electra': 'downtown\nsuburban\nuptown\noutside\ntoday',
 'roberta': 'downtown\nurban\nrural\ncity\nDowntown',
 'xlm': 'Downtown\ntown\ncity\nsouth\nChicago',
 'xlnet': 'downtown\nsuburban\never\ncity\nDowntown'}

In [ ]:
text_sentence = "In Louisiana , CIT was <mask> with repossessed ships that had supplied drilling platforms in the Gulf of Mexico "
get_all_predictions(text_sentence)

In Louisiana , CIT was <mask> with repossessed ships that had supplied drilling platforms in the Gulf of Mexico 


{'bart': 'the\nresponsible\ncharged\ninvolved\nalso',
 'bert': 'dealing\ninvolved\nworking\nassociated\noperating',
 'electra': 'equipped\narmed\nassociated\nstocked\ncombined',
 'roberta': 'working\ndealing\noperating\ncompeting\nflooded',
 'xlm': 'dealing\ninvolved\nassociated\nregistered\nworking',
 'xlnet': 'involved\ndealing\nin\nnot\nwas'}

In [ ]:
text_sentence = "Orion , a contract carrier which formerly <mask> for United Parcel Service , had agreed to replace Eastern pilots if they followed the machinists on a walkout "
get_all_predictions(text_sentence)

Orion , a contract carrier which formerly <mask> for United Parcel Service , had agreed to replace Eastern pilots if they followed the machinists on a walkout 


{'bart': 'flew\noperated\nworked\nhandled\nserved',
 'bert': 'worked\noperated\ncompeted\nsailed\nflew',
 'electra': 'worked\noperated\nran\nwas\nserved',
 'roberta': 'worked\nflew\noperated\nsubstituted\ncompeted',
 'xlm': 'worked\nran\nwas\nworks\nserved',
 'xlnet': 'worked\noperated\nworks\narranged\ncontracted'}

In [ ]:
text_sentence = "But if all sides discover self - interest in moving toward effective control of the toxics problem , instead of <mask> grimly to their familiar trenches , the results will be as remarkable as the law itself "
get_all_predictions(text_sentence)

But if all sides discover self - interest in moving toward effective control of the toxics problem , instead of <mask> grimly to their familiar trenches , the results will be as remarkable as the law itself 


{'bart': 'returning\nretreating\njust\nsimply\ncontinuing',
 'bert': 'clinging\nsticking\nholding\nreturning\nlooking',
 'electra': 'returning\nmoving\nclinging\nretreating\ngoing',
 'roberta': 'retreating\nreturning\nretiring\nsinking\nclinging',
 'xlm': 'going\nturning\nwalking\nrunning\nadding',
 'xlnet': 'clinging\nreturning\nstuck\nsticking\nattending'}

In [ ]:
text_sentence = "Real power still <mask> with Pretoria just as it does with Jerusalem"
get_all_predictions(text_sentence)

Real power still <mask> with Pretoria just as it does with Jerusalem


{'bart': 'resides\nrests\nlies\nin\nlives',
 'bert': 'rests\ncomes\nexists\nworks\ngrows',
 'electra': 'works\nexists\nplays\nis\nstays',
 'roberta': 'rests\nlies\nresides\nexists\nsits',
 'xlm': 'works\nstuck\nis\non\nmatter',
 'xlnet': 'still\nis\nremains\ntogether\nalive'}

In [ ]:
text_sentence = "Every time it <mask> , the family placed cups and bowls all over the house to catch the leaks"
get_all_predictions(text_sentence)

Every time it <mask> , the family placed cups and bowls all over the house to catch the leaks


{'bart': 'r\nrains\nwas\nleaked\nstarted',
 'bert': 'rained\nhappened\nrains\nwas\nstarted',
 'electra': 'happened\nleaked\ndid\nrained\nworked',
 'roberta': 'leaked\nrains\nhappened\nbroke\nleaks',
 'xlm': 'happened\nwas\ndid\ncame\npassed',
 'xlnet': 'was\nit\nis\nIt\nthey'}

In [ ]:
text_sentence = "And Mr. McGegan , who does n\'t mince words when he dislikes something or someone -LRB- he called the countertenor in his recent Long Beach production \'\' awful \'\' -RRB- , said he was <mask> out by Mr. Davies \'s strangely articulated yet strikingly clear conducting of Beethoven \'s \'\' Eroica \'\' Symphony"
get_all_predictions(text_sentence)

And Mr. McGegan , who does n't mince words when he dislikes something or someone -LRB- he called the countertenor in his recent Long Beach production '' awful '' -RRB- , said he was <mask> out by Mr. Davies 's strangely articulated yet strikingly clear conducting of Beethoven 's '' Eroica '' Symphony


{'bart': 'impressed\nnot\nstruck\npleased\nparticularly',
 'bert': 'put\nfreaked\ngrossed\nstruck\nhelped',
 'electra': 'freaked\ndrowned\nsingled\nstruck\nstressed',
 'roberta': 'put\nstressed\nturned\ncaught\nworn',
 'xlm': 'left\nstood\nhelped\ncalled\nbrought',
 'xlnet': 'put\nleft\nstruck\nturned\nthrown'}

In [ ]:
text_sentence = "London share prices finished a bumpy session little changed Monday , after the market <mask> on news of an unexpected rise in U.K. interest rates"
get_all_predictions(text_sentence)

London share prices finished a bumpy session little changed Monday , after the market <mask> on news of an unexpected rise in U.K. interest rates


{'bart': 'had\nopened\ntook\nfell\nrallied',
 'bert': 'turned\nbroke\nopened\nexploded\nburst',
 'electra': 'fell\nclosed\nrose\nrallied\nplunged',
 'roberta': 'rallied\nplunged\njumped\nfell\nslumped',
 'xlm': 'took\nfocused\nclosed\nreported\nturned',
 'xlnet': 'market\nclosed\nput\nwas\nwent'}

In [ ]:
text_sentence = "The virus <mask> Argonne National Laboratory outside Chicago starting at 11:54 p.m. EST Wednesday and throughout the night"
get_all_predictions(text_sentence)

The virus <mask> Argonne National Laboratory outside Chicago starting at 11:54 p.m. EST Wednesday and throughout the night


{'bart': 'will\nwas\nis\ncould\ntesting',
 'bert': 'entered\nreached\nhit\nstruck\nattacked',
 'electra': 'reached\nleft\nvisited\nentered\nhit',
 'roberta': 'struck\nhit\nreached\nentered\nattacked',
 'xlm': 'at\nin\nhit\nreached\naffected',
 'xlnet': 'at\nAt\nof\nin\non'}